#### This was copy from all other script instead of being properly organized for submission. Hence, there may be a lot of file/code has incorrect path/organization. Please use this link to go to the correct source on Github

https://github.com/ptran779/NeuroPrediction

## Analysis

In [ ]:
# Load Data

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

# prepare the data
# read in data
data = pd.read_csv("../resource/train_data.csv")
data2 = pd.read_csv("../resource/train_data2.csv")

# load in additional features for each neuron
feature_weights = pd.read_csv("../resource/feature_weights.csv")
morph_embeddings = pd.read_csv("../resource/morph_embeddings.csv")

# join all feature_weights
feature_weights["feature_weights"] = (
    feature_weights.filter(regex="feature_weight_")
    .sort_index(axis=1)
    .apply(lambda x: np.array(x), axis=1)
)
# delete the feature_weight_i columns
feature_weights.drop(
    feature_weights.filter(regex="feature_weight_").columns, axis=1, inplace=True
)

# join all morph_embed_i columns into a single np.array column
morph_embeddings["morph_embeddings"] = (
    morph_embeddings.filter(regex="morph_emb_")
    .sort_index(axis=1)
    .apply(lambda x: np.array(x), axis=1)
)
# delete the morph_embed_i columns
morph_embeddings.drop(
    morph_embeddings.filter(regex="morph_emb_").columns, axis=1, inplace=True
)

data = (
    data.merge(
        feature_weights.rename(columns=lambda x: "pre_" + x),
        how="left",
        validate="m:1",
        copy=False,
    )
    .merge(
        feature_weights.rename(columns=lambda x: "post_" + x),
        how="left",
        validate="m:1",
        copy=False,
    )
    .merge(
        morph_embeddings.rename(columns=lambda x: "pre_" + x),
        how="left",
        validate="m:1",
        copy=False,
    )
    .merge(
        morph_embeddings.rename(columns=lambda x: "post_" + x),
        how="left",
        validate="m:1",
        copy=False,
    )
)

data2 = (
    data2.merge(
        feature_weights.rename(columns=lambda x: "pre_" + x),
        how="left",
        validate="m:1",
        copy=False,
    )
    .merge(
        feature_weights.rename(columns=lambda x: "post_" + x),
        how="left",
        validate="m:1",
        copy=False,
    )
    .merge(
        morph_embeddings.rename(columns=lambda x: "pre_" + x),
        how="left",
        validate="m:1",
        copy=False,
    )
    .merge(
        morph_embeddings.rename(columns=lambda x: "post_" + x),
        how="left",
        validate="m:1",
        copy=False,
    )
)

In [ ]:
# reorganized data and remove all useless neuron -- write output to new file
linkedDat = data[data['connected'] == True]
print("Only {} out of {} are real sender".format(linkedDat['pre_nucleus_id'].nunique(), data['pre_nucleus_id'].nunique()))
print("Only {} out of {} are real receiver".format(linkedDat['post_nucleus_id'].nunique(), data['post_nucleus_id'].nunique()))

# obtain useful neuro related set (neuron that actually connect)
preNeroLinked = set(linkedDat['pre_nucleus_id'].unique())
postNeroLinked = set(linkedDat['post_nucleus_id'].unique())

# down-sample to only point that contain 2 useful neuron
usefullDat = data[(data['pre_nucleus_id']).isin(preNeroLinked) & (data['post_nucleus_id'].isin(postNeroLinked))]
print(len(usefullDat))
usefullDat.info()
usefullDat.to_csv('data2.csv')

### Visualization of all neurons position and maybe synapses. WARN: expect lag for plot 

In [ ]:

# extract only connected neuron
linkedDat = data[data['connected'] == True]

pairs1 = set()
pairs2 = {}
c = 0
for index, row in data.iterrows():
    pairs1.add((row['pre_nucleus_x'], row['pre_nucleus_y'], row['pre_nucleus_z']))
    pairs1.add((row['post_nucleus_x'], row['post_nucleus_y'], row['post_nucleus_z']))
for index, row in linkedDat.iterrows():
    tmp = (row['pre_nucleus_id'], row['post_nucleus_id'])
    if tmp in pairs2:
        c += 1
    else:
        pairs2[tmp] = (row['pre_nucleus_x'], row['post_nucleus_x'], row['pre_nucleus_y'], row['post_nucleus_y'],
                       row['pre_nucleus_z'], row['post_nucleus_z'])


def color_nodes(G):
    """Colors the nodes of a graph based on their output and input.

    Args:
        G: A networkx graph object.

    Returns:
        A dictionary mapping node IDs to colors.
    """

    colors = []
    for node in G.nodes():
        if G.out_degree(node) > 0:
            if G.in_degree(node) > 0:
                colors.append("purple")
            else:
                colors.append("red")
        else:
            colors.append("blue")
    return colors


# Map non-filter direct neuron
G1 = nx.DiGraph()
for i in pairs2:  # add all pre and post to map
    G1.add_node(i[0], XY=(pairs2[i][0], pairs2[i][2]), XZ=(pairs2[i][0], pairs2[i][4]))
    G1.add_node(i[1], XY=(pairs2[i][1], pairs2[i][3]), XZ=(pairs2[i][1], pairs2[i][5]))
    G1.add_edge(i[0], i[1])

x = []
y = []
z = []
for i in pairs1:
    x.append(i[0])
    y.append(i[1])
    z.append(i[2])

plt.figure(figsize=(10,5))
plt.scatter(x, y, color='pink')
colors = color_nodes(G1)
XY = nx.get_node_attributes(G1, 'XY')
nx.draw(G1, pos=XY, node_color=colors, node_size=20)
plt.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
plt.axis(True)
plt.xlabel("x (nm)")
plt.ylabel("y (nm)")
plt.show()

plt.figure(figsize=(10,5))
plt.scatter(x, y, color='pink')
XY = nx.get_node_attributes(G1, 'XY')
nx.draw_networkx_nodes(G1, pos=XY, node_color=colors, node_size=20)
plt.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
plt.axis(True)
plt.xlabel("x (nm)")
plt.ylabel("y (nm)")
plt.show()

plt.figure(figsize=(10,5))
XY = nx.get_node_attributes(G1, 'XY')
nx.draw_networkx_nodes(G1, pos=XY, node_color=colors, node_size=20)
plt.scatter(linkedDat.dendritic_coor_x.values, linkedDat.dendritic_coor_y.values, c='orange')
plt.tick_params(left=True, bottom=True, labelleft=True, labelbottom=True)
plt.axis(True)
plt.xlabel("x (nm)")
plt.ylabel("y (nm)")
plt.show()


plt.figure(figsize=(10,5))
XY = nx.get_node_attributes(G1, 'XY')
plt.scatter(data.dendritic_coor_x.values, data.dendritic_coor_y.values, c='yellow')
plt.scatter(linkedDat.dendritic_coor_x.values, linkedDat.dendritic_coor_y.values, c='orange')
plt.axis(True)
plt.xlabel("x (nm)")
plt.ylabel("y (nm)")
plt.show()

x1 = []
y1 = []
c1 = []
# crazy loop
for index, row in linkedDat.iterrows():
    x1.append(row.pre_nucleus_x)
    x1.append(row.post_nucleus_x)
    y1.append(row.pre_nucleus_y)
    y1.append(row.post_nucleus_y)
    if row.pre_brain_area == "RL": c1.append("red")
    elif row.pre_brain_area == "AL": c1.append("blue")
    else: c1.append("green")

    if row.post_brain_area == "RL": c1.append("red")
    elif row.post_brain_area == "AL": c1.append("blue")
    else: c1.append("green")

plt.figure(figsize=(10,5))
plt.scatter(x1, y1, color=c1)
plt.show()
# plt.figure()
# plt.scatter(x, z, color='pink')
# colors = color_nodes(G1)
# XZ = nx.get_node_attributes(G1, 'XZ')
# # nx.draw(G1, pos=XZ, node_color=colors, node_size=20)
# plt.show()

allneuron = data2[['pre_nucleus_id', 'pre_nucleus_x', 'pre_nucleus_y', 'pre_nucleus_z',
                   'post_nucleus_id', 'post_nucleus_x', 'post_nucleus_y', 'post_nucleus_z']]
# allneuronid = np.concatenate([allneuron.pre_nucleus_id.unique(), allneuron.post_nucleus_id.unique()])

### PCA & other analysis

In [ ]:
from sklearn.impute import SimpleImputer
import ast  # Module to safely evaluate literal strings as Python expressions

# Assuming 'data' is your DataFrame
column_to_impute = 'pre_morph_embeddings'

data[column_to_impute].fillna(0, inplace=True)

data.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# List of categorical columns
categorical_columns = ['compartment', 'pre_brain_area','post_brain_area']

# Apply label encoding to each categorical column
for column in categorical_columns:
    data[column] = label_encoder.fit_transform(data[column])

# Display the updated DataFrame
data.info()

In [ ]:
# Calculating the Eulidean distances between the axonal and dendritic coordinates



# Calculate Euclidean distances and create a new column 'euclidean_distance'
data['euclidean_distance'] = np.sqrt(
    (data['axonal_coor_x'] - data['dendritic_coor_x'])**2 +
    (data['axonal_coor_y'] - data['dendritic_coor_y'])**2 +
    (data['axonal_coor_z'] - data['dendritic_coor_z'])**2
)

# The 'euclidean_distance' column now contains the Euclidean distances

# Calculate Euclidean distances between axonal and dendritic coordinates for both pre and post neurons
data['axonal_distance_pre'] = np.sqrt((data['axonal_coor_x'] - data['dendritic_coor_x'])**2 + (data['axonal_coor_y'] - data['dendritic_coor_y'])**2)
data['axonal_distance_post'] = np.sqrt((data['axonal_coor_x'] - data['dendritic_coor_x'])**2 + (data['axonal_coor_y'] - data['dendritic_coor_y'])**2)

data['dendritic_distance_pre'] = np.sqrt((data['dendritic_coor_x'] - data['axonal_coor_x'])**2 + (data['dendritic_coor_y'] - data['axonal_coor_y'])**2)
data['dendritic_distance_post'] = np.sqrt((data['dendritic_coor_x'] - data['axonal_coor_x'])**2 + (data['dendritic_coor_y'] - data['axonal_coor_y'])**2)

# Combine these distances with Euclidean distances between readout locations
data['euclidean_distance_pre'] = np.sqrt((data['pre_rf_x'] - data['post_rf_x'])**2 + (data['pre_rf_y'] - data['post_rf_y'])**2)
data['euclidean_distance_post'] = np.sqrt((data['pre_rf_x'] - data['post_rf_x'])**2 + (data['pre_rf_y'] - data['post_rf_y'])**2)

# Dataset contains both structural (axonal and dendritic distances) and spatial (Euclidean distances) features.


# Calculate the relative positions of post-synaptic neurons with respect to pre-synaptic neurons
data['relative_position_x'] = data['post_rf_x'] - data['pre_rf_x']
data['relative_position_y'] = data['post_rf_y'] - data['pre_rf_y']

# dataset contains "relative_position_x" and "relative_position_y" features

# Create binary directional features
data['is_left'] = (data['relative_position_x'] < 0).astype(int)
data['is_right'] = (data['relative_position_x'] > 0).astype(int)
data['is_above'] = (data['relative_position_y'] < 0).astype(int)
data['is_below'] = (data['relative_position_y'] > 0).astype(int)

# Dataset contains binary directional features

# Create interaction terms between rf_x and rf_y for pre-synaptic neurons
data['pre_rf_x_rf_y_interaction'] = data['pre_rf_x'] * data['pre_rf_y']

# Create interaction terms between rf_x and rf_y for post-synaptic neurons
data['post_rf_x_rf_y_interaction'] = data['post_rf_x'] * data['post_rf_y']

# The dataset now contains interaction terms that capture relationships in visual space

In [ ]:
#Processing the morph embeddings
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data['pre_morph_embeddings'] = data['pre_morph_embeddings'].apply(lambda x: scaler.fit_transform(np.array(x).reshape(-1, 1)))
data['post_morph_embeddings'] = data['post_morph_embeddings'].apply(lambda x: scaler.fit_transform(np.array(x).reshape(-1, 1)))

def flatten_arrays(arrays):
    return [item for sublist in arrays for item in sublist]

data['pre_morph_embeddings'] = data['pre_morph_embeddings'].apply(flatten_arrays)
data['post_morph_embeddings'] = data['post_morph_embeddings'].apply(flatten_arrays)
data['pre_morph_embeddings']

In [ ]:
# PCA on morph embeddings
from sklearn.decomposition import PCA
import numpy as np

# Ensure all arrays in 'pre_morph_embeddings' and 'post_morph_embeddings' have the same size
data['pre_morph_embeddings'] = data['pre_morph_embeddings'].apply(lambda x: np.array(x) if len(x) == 32 else np.array([0.0] * 32))
data['post_morph_embeddings'] = data['post_morph_embeddings'].apply(lambda x: np.array(x) if len(x) == 32 else np.array([0.0] * 32))

# Apply PCA
pca = PCA(n_components=2)
data['pre_morph_embeddings'] = list(pca.fit_transform(np.vstack(data['pre_morph_embeddings'])))
data['post_morph_embeddings'] = list(pca.fit_transform(np.vstack(data['post_morph_embeddings'])))
data['post_morph_embeddings']

In [ ]:
# PCA on feature weights
scaler = StandardScaler()
data['pre_feature_weights'] = data['pre_feature_weights'].apply(lambda x: scaler.fit_transform(np.array(x).reshape(-1, 1)))
data['post_feature_weights'] = data['post_feature_weights'].apply(lambda x: scaler.fit_transform(np.array(x).reshape(-1, 1)))


def flatten_arrays(arrays):
    return [item for sublist in arrays for item in sublist]

data['pre_feature_weights'] = data['pre_feature_weights'].apply(flatten_arrays)
data['post_feature_weights'] = data['post_feature_weights'].apply(flatten_arrays)


# Apply PCApost_feature_weights
pca = PCA(n_components=2)
data['pre_feature_weights'] = list(pca.fit_transform(np.vstack(data['pre_feature_weights'])))
data['post_feature_weights'] = list(pca.fit_transform(np.vstack(data['post_feature_weights'])))



# # Extract feature weight columns
# feature_weight_columns = ['pre_feature_weights', 'post_feature_weights']

# # Standardize the data
# scaler = StandardScaler()
# data[feature_weight_columns] = scaler.fit_transform(data[feature_weight_columns])

# # Create a PCA object
# pca = PCA(n_components=2)
# data_pca = pca.fit_transform(data[feature_weight_columns])

In [ ]:
#Kurtosis, skew, stats - feature weights
from scipy.stats import skew, kurtosis

data['pre_feature_weights_sum'] = data['pre_feature_weights'].apply(lambda x: sum(x))
data['post_feature_weights_sum'] = data['post_feature_weights'].apply(lambda x: sum(x))
data['feature_weights_difference'] = abs(data['pre_feature_weights_sum'] - data['post_feature_weights_sum'])

data['pre_feature_weights_variance'] = data['pre_feature_weights'].apply(lambda x: np.var(x))
data['post_feature_weights_variance'] = data['post_feature_weights'].apply(lambda x: np.var(x))


data['pre_feature_weights_skewness'] = data['pre_feature_weights'].apply(lambda x: skew(x))
data['post_feature_weights_skewness'] = data['post_feature_weights'].apply(lambda x: skew(x))
data['pre_feature_weights_kurtosis'] = data['pre_feature_weights'].apply(lambda x: kurtosis(x))
data['post_feature_weights_kurtosis'] = data['post_feature_weights'].apply(lambda x: kurtosis(x))

In [ ]:
# Skew, stats, kurt for Morph embeddings
from scipy.stats import skew, kurtosis
import numpy as np


# Sum the morphological embeddings for pre and post neurons
data['pre_morph_embeddings_sum'] = data['pre_morph_embeddings'].apply(lambda x: np.sum(x, axis=0))
data['post_morph_embeddings_sum'] = data['post_morph_embeddings'].apply(lambda x: np.sum(x, axis=0))

# Calculate the Euclidean distance between summed morphological embeddings
data['morph_embeddings_distance'] = data.apply(lambda row: np.linalg.norm(row['pre_morph_embeddings_sum'] - row['post_morph_embeddings_sum']), axis=1)

# Calculate the variance of morphological embeddings
data['pre_morph_embeddings_variance'] = data['pre_morph_embeddings'].apply(lambda x: np.var(x, axis=0))
data['post_morph_embeddings_variance'] = data['post_morph_embeddings'].apply(lambda x: np.var(x, axis=0))

# Calculate the skewness and kurtosis of morphological embeddings
data['pre_morph_embeddings_skewness'] = data['pre_morph_embeddings'].apply(lambda x: skew(x, axis=0))
data['post_morph_embeddings_skewness'] = data['post_morph_embeddings'].apply(lambda x: skew(x, axis=0))
data['pre_morph_embeddings_kurtosis'] = data['pre_morph_embeddings'].apply(lambda x: kurtosis(x, axis=0))
data['post_morph_embeddings_kurtosis'] = data['post_morph_embeddings'].apply(lambda x: kurtosis(x, axis=0))

In [ ]:
# PCA with Explained variance on whole data set
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


# Exclude non-numeric and non-categorical columns from PCA
exclude_cols = ['ID',
                'pre_feature_weights',
                'post_feature_weights',
                'pre_morph_embeddings',
                'post_morph_embeddings',
                'projection_group',
                'connected',
                'is_left',
                'is_right',
                'is_above',
                'is_below'
               ]

# Select only numeric columns for PCA
numeric_columns = [col for col in data.columns if col not in exclude_cols]

# Standardize the data (if needed)
scaler = StandardScaler()
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

# Create a PCA object
pca = PCA()

# Fit PCA and transform the data
pca_result = pca.fit_transform(data[numeric_columns])

# Calculate explained variance
explained_variance = pca.explained_variance_ratio_

# Cumulative explained variance
cumulative_explained_variance = explained_variance.cumsum()

# Visualize explained variance
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(explained_variance) + 1), cumulative_explained_variance, marker='o', linestyle='-', color='b')
plt.title('Explained Variance with PCA')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.grid(True)
plt.show()


In [ ]:
# Apply PCA to data
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Exclude non-numeric and non-categorical columns from the PCA
exclude_cols = ['ID',
                'pre_feature_weights',
                'post_feature_weights',
                'pre_morph_embeddings',
                'post_morph_embeddings',
                'projection_group',
                'connected',
                'is_left',
                'is_right',
                'is_above',
                'is_below'
               ]

# Select only numeric columns for PCA
numeric_columns = [col for col in data.columns if col not in exclude_cols]
pca_num_of_components = 30

# Standardize the data (if needed)
scaler = StandardScaler()
data[numeric_columns] = scaler.fit_transform(data[numeric_columns])

# Create a PCA object (you can specify the number of components)
pca = PCA(n_components=pca_num_of_components)

# Fit and transform the data
pca_result = pca.fit_transform(data[numeric_columns])

# Create a DataFrame for visualization
pca_df = pd.DataFrame(data=pca_result, columns=[col for col in range(pca_num_of_components)])

## K means clustering

In [ ]:
from sklearn.cluster import KMeans

# Select only numeric columns for KMeans clustering
numeric_columns_for_clustering = [col for col in pca_df.columns if col != 'Cluster']

num_clusters = 2

# Create a KMeans object
kmeans = KMeans(n_clusters=num_clusters, random_state=42)

# Fit the model to the PCA results
kmeans.fit(pca_df[numeric_columns_for_clustering])

# Add the cluster labels to the PCA DataFrame
pca_df['Cluster'] = kmeans.labels_

# Visualize the clusters in the scatter plot
plt.figure(figsize=(10, 6))
for cluster in range(num_clusters):
    cluster_data = pca_df[pca_df['Cluster'] == cluster]
    plt.scatter(cluster_data[0], cluster_data[1], label=f'Cluster {cluster}', alpha=0.7)

plt.title('K-means Clustering after PCA')
plt.xlabel('Principal Component 1 (PC1)')
plt.ylabel('Principal Component 2 (PC2)')
plt.legend()
plt.grid(True)
plt.show()

## DNN

### Datamanager: handle the input data and prepare all preprocessing for selected DNN model

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

class DataManager:
    """
    Handle DNN dataset and apply modification needed
    """
    def __init__(self):
        self._dat_main = None
        self._dat_lboard = None
        self._dat_feat = None
        self._dat_morp = None
        self.BATCH_SIZE = 0
        self.EPOCHS = 0
        self.BUFFER_SIZE = 0
        self._train_ds = None
        self._val_ds = None
        self._test_ds = None
        self.step_per_epoch = None
        self.lea_ds = None  # wip. just a bunch of feature for now
        self.pre_morph_pca = None
        self.post_morph_pca = None

    def config(self, batch_size: int, epochs: int, buffer_size: int):
        self.BATCH_SIZE = batch_size
        self.EPOCHS = epochs
        self.BUFFER_SIZE = buffer_size

    def load_dat_feat(self, fname: str):
        self._dat_feat = pd.read_csv(fname)
        # join all morph_embed_i columns into a single np.array column
        self._dat_feat["feature_weights"] = (
            self._dat_feat.filter(regex="feature_weight_")
            .sort_index(axis=1)
            .apply(lambda x: np.array(x), axis=1)
        )
        self._dat_feat.drop(self._dat_feat.filter(regex="feature_weight_").columns, axis=1, inplace=True)

    def load_dat_morp(self, fname: str):
        self._dat_morp = pd.read_csv(fname)
        self._dat_morp["morph_embeddings"] = (
            self._dat_morp.filter(regex="morph_emb_")
            .sort_index(axis=1)
            .apply(lambda x: np.array(x), axis=1)
        )
        self._dat_morp.drop(self._dat_morp.filter(regex="morph_emb_").columns, axis=1, inplace=True)

    def load_dat_main(self, fname: str):
        if not self.dataIsLoadedDat():
            raise Exception("please load morp and feature data first")
        self._dat_main = pd.read_csv(fname)

        self._dat_main = (
            self._dat_main.merge(
                self._dat_feat.rename(columns=lambda x: "pre_" + x),
                how="left",
                validate="m:1",
                copy=False,
            )
            .merge(
                self._dat_feat.rename(columns=lambda x: "post_" + x),
                how="left",
                validate="m:1",
                copy=False,
            )
            .merge(
                self._dat_morp.rename(columns=lambda x: "pre_" + x),
                how="left",
                validate="m:1",
                copy=False,
            )
            .merge(
                self._dat_morp.rename(columns=lambda x: "post_" + x),
                how="left",
                validate="m:1",
                copy=False,
            )
        )

        dm1, dm2 = len(self._dat_main['pre_morph_embeddings'][0]), len(self._dat_main['post_morph_embeddings'][0])
        for i, r in self._dat_main.iterrows():
            if isinstance(r['pre_morph_embeddings'], float):
                self._dat_main.at[i, 'pre_morph_embeddings'] = np.full(dm1, 0.001)
            if isinstance(r['post_morph_embeddings'], float):
                self._dat_main.at[i, 'post_morph_embeddings'] = np.full(dm2, 0.001)

    def load_dat_lboard(self, fname: str):
        if not self.dataIsLoadedDat():
            raise Exception("please load morp and feature data first")

        self._dat_lboard = pd.read_csv(fname)

        self._dat_lboard = (
            self._dat_lboard.merge(
                self._dat_feat.rename(columns=lambda x: "pre_" + x),
                how="left",
                validate="m:1",
                copy=False,
            )
            .merge(
                self._dat_feat.rename(columns=lambda x: "post_" + x),
                how="left",
                validate="m:1",
                copy=False,
            )
            .merge(
                self._dat_morp.rename(columns=lambda x: "pre_" + x),
                how="left",
                validate="m:1",
                copy=False,
            )
            .merge(
                self._dat_morp.rename(columns=lambda x: "post_" + x),
                how="left",
                validate="m:1",
                copy=False,
            )
        )

        dm1, dm2 = len(self._dat_lboard['pre_morph_embeddings'][0]), len(self._dat_lboard['post_morph_embeddings'][0])
        for i, r in self._dat_lboard.iterrows():
            if isinstance(r['pre_morph_embeddings'], float):
                self._dat_lboard.at[i, 'pre_morph_embeddings'] = np.full(dm1, 0.001)
            if isinstance(r['post_morph_embeddings'], float):
                self._dat_lboard.at[i, 'post_morph_embeddings'] = np.full(dm2, 0.001)

    def dataIsLoadedDat(self):
        return not (self._dat_morp is None or self._dat_feat is None)

    def prepTrainValTest(self, w_trn, w_val, w_tst, random_state=0):
        # split data
        tmp = w_tst/(w_trn+w_val+w_tst)
        dm_df, test_df = train_test_split(self._dat_main, test_size=tmp, random_state=random_state)
        tmp = w_val/(w_val+w_trn)
        train_df, val_df = train_test_split(dm_df, test_size=tmp, random_state=random_state)

        # process
        f_trn, l_trn = self.processFromDf(train_df, trainmode=True)
        f_val, l_val = self.processFromDf(val_df, trainmode=False)
        f_tst, l_tst = self.processFromDf(test_df, trainmode=False)

        # split positive and negative sample and make them
        pos_ds = self.makeDs(f_trn, l_trn, np.where(l_trn == 1))
        neg_ds = self.makeDs(f_trn, l_trn, np.where(l_trn == 0))
        resampled_ds = tf.data.Dataset.sample_from_datasets([pos_ds, neg_ds], weights=[.5, .5])

        self._train_ds = resampled_ds.batch(self.BATCH_SIZE).prefetch(2)
        # self._train_ds = self.makeDs(f_trn, l_trn)  # test without rebalancing
        # self._train_ds = pos_ds.batch(self.BATCH_SIZE).prefetch(2)  # train with just positive to offset the balance
        self.step_per_epoch = np.ceil(2.0 * (len(l_trn) - np.count_nonzero(l_trn)) / self.BATCH_SIZE)

        self._val_ds = self.makeDs(f_val, l_val)
        self._test_ds = self.makeDs(f_tst, l_tst)

    def loadLeader(self):
        self.lea_ds, _ = self.processFromDf(self._dat_lboard, skip_y=1, trainmode=False)

    def getTrainDs(self):
        return self._train_ds
    
    def getValDs(self):
        return self._val_ds
    
    def getTestDs(self):
        return self._test_ds

    def getLeadDS(self):
        return self.lea_ds

    @staticmethod
    def oneHotConverter(raw):
        """
        convert a 1D array to one-hot-coding
        :param raw: an array
        :return: 2D matrix
        """
        cls = np.unique(raw)
        out = np.zeros([len(raw), len(cls)])
        cls_look_up = {item: index for index, item in enumerate(cls)}  # convert array to dict
        for c, i in enumerate(raw):
            out[c, cls_look_up[i]] = 1  # mark target code to 1
        return out

    @staticmethod
    def cosin_feature_similarity(df, feat1, feat2):
        # perform cosin similarity on 2 feature and return array
        pre = np.vstack(df[feat1].values)
        post = np.vstack(df[feat2].values)
        out = np.zeros(len(pre))
        for c, _ in enumerate(out):
            out[c] = np.dot(pre[c], post[c])/(np.linalg.norm(pre[c]) * np.linalg.norm(post[c]))
        return out

    def processFromDf(self, raw_df, skip_y=0, trainmode=True):
        # engineering features
        raw_df['fweight_cosin'] = DataManager.cosin_feature_similarity(
            raw_df, 'pre_feature_weights', 'post_feature_weights')
        raw_df['morp_cosin'] = DataManager.cosin_feature_similarity(
            raw_df, 'pre_morph_embeddings', 'post_morph_embeddings')

        raw_df['rf_dist'] = np.linalg.norm(raw_df[['pre_rf_x', 'pre_rf_y']].values -
                                           raw_df[['post_rf_x', 'post_rf_y']].values, axis=1)

        raw_df['neural_displacement_x'] = (raw_df.post_nucleus_x - raw_df.pre_nucleus_x) / 100000
        raw_df['neural_displacement_y'] = (raw_df.post_nucleus_y - raw_df.pre_nucleus_y) / 100000
        raw_df['neural_displacement_z'] = (raw_df.post_nucleus_z - raw_df.pre_nucleus_z) / 100000


        indat1 = np.copy(raw_df[['adp_dist', 'pre_oracle', 'post_oracle', 'pre_skeletal_distance_to_soma',
                                 'post_skeletal_distance_to_soma', 'pre_test_score', 'post_test_score', 'fweight_cosin',
                                 'morp_cosin', 'rf_dist', 'neural_displacement_x', 'neural_displacement_y',
                                 'neural_displacement_z']].values)
        # brain area
        raw = raw_df.pre_brain_area.values
        indat2 = DataManager.oneHotConverter(raw)
        raw = raw_df.post_brain_area.values
        indat3 = DataManager.oneHotConverter(raw)
        # compartment
        raw = raw_df.compartment.values
        indat4 = DataManager.oneHotConverter(raw)

        # morph embedding
        indat5 = np.vstack(raw_df.pre_morph_embeddings.values)
        indat6 = np.vstack(raw_df.post_morph_embeddings.values)
        # if trainmode:
        #     pca = PCA(n_components=8)
        #     indat5 = pca.fit_transform(np.vstack(raw_df.pre_morph_embeddings))
        #     self.pre_morph_pca = pca.components_
        #     indat6 = pca.fit_transform(np.vstack(raw_df.post_morph_embeddings))
        #     self.post_morph_pca = pca.components_
        # else:
        #     indat5 = np.dot(np.vstack(raw_df.pre_morph_embeddings), self.pre_morph_pca.T)
        #     indat6 = np.dot(np.vstack(raw_df.post_morph_embeddings), self.post_morph_pca.T)

        # feature weight
        # indat7 = np.stack(raw_df.pre_feature_weights)
        # indat8 = np.stack(raw_df.post_feature_weights)

        # indat7 = pca_pre_feature_weight_result
        # indat8 = pca_post_feature_weight_result

        labels = None
        if not skip_y:
            raw = raw_df.connected.values
            labels = DataManager.oneHotConverter(raw)[:, 1]

        # Normalization
        # transform bulk data  -- try forcing to magnitude of 10
        arr = np.transpose(indat1)
        arr[0] = arr[0] / 5000  # scale down large value
        # arr[1] = arr[1] * 10  # upscale
        # arr[2] = arr[2] * 10  # upscale
        arr[3] = arr[3] / 2000000  # scale down large value
        arr[4] = arr[4] / 2000000  # scale down large value
        arr[9] = arr[9] / 30
        return (indat1, indat2, indat3, indat4, indat5, indat6), labels

    def makeDs(self, features_list, labels, index=None):
        """
        :param features_list: [feature1, feature2, ...]
        :param labels: output label
        :param index: position for this dataset -- training need index to split pos and neg sample for rabalancing, 
        val and test does not. I know it's wrong in every shape and form, but I'm lazy
        :return: tf.dataset
        """
        used_features = {}
        ds = None
        if index is None:
            for c, feature in enumerate(features_list):
                used_features["Feature {}".format(c)] = feature
            ds = tf.data.Dataset.from_tensor_slices((used_features, labels)).cache()
            ds = ds.batch(self.BATCH_SIZE).prefetch(2)
        else:
            for c, feature in enumerate(features_list):
                used_features["Feature {}".format(c)] = feature[index]
            ds = tf.data.Dataset.from_tensor_slices((used_features, labels[index]))
            ds = ds.shuffle(self.BUFFER_SIZE).repeat()
        return ds

### NeroClassifier: DNN model used to detect synapses

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.utils import plot_model
from DataManager import *
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, balanced_accuracy_score

SEED = 7
BATCH_SIZE = 16
EPOCHS = 200
BUFFER_SIZE = 10000
STEP_PER_EPOCH = 32
TRN = 0.8
VAL = 0.1
TST = 0.1

METRICS = [
    keras.metrics.BinaryCrossentropy(name='cross entropy'),  # same as model's loss
    keras.metrics.MeanSquaredError(name='MSE'),
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'),
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    keras.metrics.AUC(name='prc', curve='PR'),  # precision-recall curve
]
checkpoint_path_store = 'resource/checkpoint/dum.ckpt'

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_auc', verbose=1, patience=10, mode='max',
                                                  restore_best_weights=True)
keras.utils.set_random_seed(SEED)
# activate GPU accelerator
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')  # Use the first GPU


cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path_store, save_weights_only=True, save_freq=5)


def plot_metrics(history):
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    metrics = ['loss', 'prc', 'precision', 'recall']
    for n, metric in enumerate(metrics):
        name = metric.replace("_", " ").capitalize()
        plt.subplot(2, 2, n + 1)
        plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
        plt.plot(history.epoch, history.history['val_' + metric],
                 color=colors[1], linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        if metric == 'loss':
            plt.ylim([0, plt.ylim()[1]])
        elif metric == 'auc':
            plt.ylim([0.8, 1])
        else:
            plt.ylim([0, 1])

        plt.legend()


# Input Data
datamanager = DataManager()
datamanager.config(BATCH_SIZE, EPOCHS, BUFFER_SIZE)
datamanager.load_dat_feat("./resource/feature_weights.csv")
datamanager.load_dat_morp("./resource/morph_embeddings.csv")
datamanager.load_dat_main("./resource/train_data2.csv")
datamanager.prepTrainValTest(TRN, VAL, TST, SEED)
# Create a model
in0 = keras.layers.Input(shape=(13,), dtype='float32', name='Feature 0')
in1 = keras.layers.Input(shape=(3,), dtype='float32', name='Feature 1')
in2 = keras.layers.Input(shape=(3,), dtype='float32', name='Feature 2')
in3 = keras.layers.Input(shape=(7,), dtype='float32', name='Feature 3')
in4 = keras.layers.Input(shape=(32,), dtype='float32', name='Feature 4')
in5 = keras.layers.Input(shape=(32,), dtype='float32', name='Feature 5')
# in6 = keras.layers.Input(shape=(512,), dtype='float32', name='Feature 6')
# in7 = keras.layers.Input(shape=(512,), dtype='float32', name="Feature 7")
layer = keras.layers.Concatenate(axis=1)([in0, in1, in2, in3, in4, in5])
layer = keras.layers.Dense(64, activation=keras.activations.gelu)(layer)
layer = keras.layers.Dropout(0.25)(layer)
layer = keras.layers.Dense(64, activation=keras.activations.gelu)(layer)
layer = keras.layers.Dropout(0.25)(layer)
layer = keras.layers.Dense(32, activation=keras.activations.gelu)(layer)
# drop3 = keras.layers.Dropout(0.2)(dense3)
output = keras.layers.Dense(1, activation=keras.activations.sigmoid)(layer)
model = keras.models.Model([in0, in1, in2, in3, in4, in5], output)

plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)  # Set the constant learning rate
loss = tf.keras.losses.BinaryFocalCrossentropy(
    apply_class_balancing=False, alpha=0.5, gamma=8.0)

model.compile(optimizer=optimizer, loss=loss, metrics=METRICS)  # Using binary cross-entropy loss

In [ ]:
# load store weight if want to continue
checkpoint_path_load = 'resource/checkpoint/smater5.ckpt'
# model.load_weights(checkpoint_path_load)

In [ ]:
# Train the model
history = model.fit(
    datamanager.getTrainDs(),
    epochs=EPOCHS,
    steps_per_epoch=STEP_PER_EPOCH,
    # steps_per_epoch=datamanager.step_per_epoch,
    callbacks=[early_stopping, cp_callback],
    validation_data=(datamanager.getValDs()))
plot_metrics(history)

hist_val = model.evaluate(datamanager.getValDs())
print("[{} {}]\n[{} {}]".format(hist_val[3], hist_val[4], hist_val[5], hist_val[6]))
print((hist_val[3] / (hist_val[3] + hist_val[6]) + hist_val[5] / (hist_val[5] + hist_val[4])) / 2)

In [ ]:
# only run this when you're ready for test
hist_test = model.evaluate(datamanager.getTestDs())
print("[{} {}]\n[{} {}]".format(hist_test[3], hist_test[4], hist_test[5], hist_test[6]))
print((hist_test[3] / (hist_test[3] + hist_test[6]) + hist_test[5] / (hist_test[5] + hist_test[4])) / 2)

In [ ]:
# leaderboard
datamanager.load_dat_lboard('./resource/leaderboard_data.csv')
datamanager.loadLeader()
pred = model.predict(datamanager.lea_ds)
lead = datamanager._dat_lboard
lead['connected'] = pred >= 0.5
submission_data = lead.filter(['ID', 'connected'])
submission_data.to_csv('submission_data_dnn_report3.csv', index=False)

In [ ]:
# y_true = np.concatenate([y.numpy() for x, y in datamanager.getTestDs()], axis=0)
# y_prop = model.predict(datamanager.getTestDs())
# dum = []
# dumx = []
# for i in range(20, 80):
#     dumx.append(i/100)
#     y_pred = (y_prop >= i/100).astype(float)
#     dum.append(balanced_accuracy_score(y_true, y_pred))
# plt.figure()
# plt.plot(dumx, dum)
# plt.show()
# 
# # some stupid test
# a = np.array([])
# b = np.array([])
# a = a == 1
# thres = 0.6
# 
# confusion_matrix(a, b>thres)

